In [1]:
import pickle
from keras.models import load_model

# load mapping files
with open("char_to_int.pickle","rb") as file:
    char_to_int = pickle.load(file)

with open("int_to_char.pickle","rb") as file:
    int_to_char = pickle.load(file)

# load text file
with open("verwandlung.txt", encoding="utf_8", mode="r") as file: # make sure its utf_8 encoding for german texts
    contents = file.read()
    
full_text = "\n".join(contents.split("\n")[59:1952])

# load model weights
model = load_model("LSTM_weights.hdf5")

Using TensorFlow backend.
c:\users\florian\.conda\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\florian\.conda\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\florian\.conda\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\florian\.conda\envs\dl\lib\site-packages\tensorflo

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               100864    
_________________________________________________________________
dense_2 (Dense)              (None, 68)                8772      
Total params: 109,636
Trainable params: 109,636
Non-trainable params: 0
_________________________________________________________________


In [3]:
## Text generation: Pick random 40 chars from text. Let the network predict the missing char

# Choose 40 chars
line = full_text[100:140]

# Map to int
int_line = [char_to_int[c] for c in line]

# Encode as one hot array
from keras.utils import to_categorical
int_line = to_categorical(int_line, num_classes = len(set(full_text)))
line_input = int_line.reshape(1,40,68)

# Feed to Network
prediction = model.predict(x=line_input)
print(prediction)

[[3.90328079e-01 8.12467560e-03 4.36414790e-04 1.69555633e-03
  2.65258933e-07 2.48760529e-07 1.82345342e-02 1.16322731e-07
  3.68886475e-07 1.36801030e-03 3.50967554e-07 5.24626500e-07
  1.35812229e-06 6.89395529e-04 3.19102946e-06 1.05982821e-04
  1.88128763e-06 3.68303663e-05 5.57894782e-06 8.36279739e-08
  1.40827805e-01 5.76333434e-07 7.07984782e-07 7.55087845e-03
  1.27158416e-02 6.18360122e-04 8.27565818e-05 1.04767375e-03
  1.04893245e-01 1.14333750e-06 2.65955401e-04 9.51410737e-04
  3.79850292e-07 2.57529649e-07 2.42548867e-05 2.71833087e-05
  5.71845646e-07 3.61499104e-07 8.77789645e-08 6.15184348e-08
  4.35526110e-03 2.16815079e-06 3.74408364e-02 5.50805235e-05
  2.92668545e-07 1.09065818e-02 1.36260397e-07 1.49910909e-03
  2.17042374e-03 3.65512110e-02 8.67891981e-09 6.81900383e-06
  1.52631298e-01 2.50995101e-04 2.90771073e-04 2.22781990e-02
  2.61840864e-06 1.11569101e-02 3.74342153e-06 4.72091369e-06
  2.18393197e-05 3.03089321e-02 6.68498558e-07 3.49728651e-07
  2.2430

In [4]:
# Map prediciton to char
import numpy as np

prediction = int_to_char[np.argmax(prediction[0])]

In [5]:
print(line)

ungeheueren Ungeziefer verwandelt. Er la


In [6]:
full_text[130:160] # Prediciton is wrong. Doesnt matter go with it and see what the network predicts

'elt. Er lag\nauf seinem panzera'

In [8]:
## Task: Predict more than one char

# What we have: "ungeheueren Ungeziefer verwandelt. Er la" + prediction "n"
# Next step is to go with the prediction: "(remove first char)ngeheueren Ungeziefer verwandelt. Er lan(add prediction)"

line = full_text[100:140]


# Since we need these steps over and over again create prediction function
def predict_char(line):

    # Map to int
    int_line = [char_to_int[c] for c in line]

    # Encode as one hot array
    int_line = to_categorical(int_line, num_classes = len(set(full_text)))
    line_input = int_line.reshape(1,40,68)

    # Feed to Network
    prediction = model.predict(x=line_input)
    # prediction = int_to_char[np.argmax(prediction[0])] Always picking the best of argmax leads to a loop in the output text
    # Workaround is maybe to choose best out of 3 or sth.  
    prediction = prediction[0]
    prediction = np.random.choice(len(set(full_text)), 1, p=prediction)[0]
    prediction = int_to_char[prediction]
    
    return prediction

pred_counter = 1000 # how many words do you want to predict?
i=0
old_line = full_text[(100):(140)]  
print(old_line, end="")

while i < pred_counter:
    
    prediction = predict_char(line)
    line = line[1:] + prediction # line[1:] drops first element of the string
    
    i+=1
    print(prediction, end="")


ungeheueren Ungeziefer verwandelt. Er laußer sie zi mindeß unden dorf nicht vor ir fürde
besondeismafter,
denn umingon dach nicht
stall ausg mit ersosseustelen, vernerst maßte
und venderter dienen die alle, as hreöten werter, sich immährten verfulten allig,« racht inleingoher verder?« dural underte geöneskans beglieber und
dur
immürden und däß dieser Krüßtr,
ander dab is des Verpaldere Blteb verun beit verhattebstrauß, die war, durdesching
einerzungeiderfttglaugen köfnange, die Fanden Horen ja hrendistreit  auswofnnen der danon um,
der Varer umrächtlich brarande ihn wogen sich?« rach joderbeinen Zeit übersichtstübern umdegenden in die Mitter ge ulüten und; umschrißte Seinenewer. 
Ann die Gregor denn sichen Loben Vürlics vessellig stüber patrist bes was den Vo erwaum sie arst un dieremit auch von se or dien aufzurer,«
speden Kohm der seinen Gesohal, augengebogam glesches nächten im eroter; Ihlen
Verschoftre vergespeo vernwonn, däßte er dast daß, erstont dochtstendien auch fielerwurden,«